In [ ]:
!wget —load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget —quiet —save-cookies ~/cookies.txt —keep-session-cookies —no-check-certificate 'https://docs.google.com/uc?export=download&id=1a9XB3r83ZCFWLOHBp8ooz3zQFl9rEIei' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1a9XB3r83ZCFWLOHBp8ooz3zQFl9rEIei" -O open.zip && rm -rf ~/cookies.txt

--2023-08-15 16:52:23--  http://xn--quiet-3u3b/
Resolving xn--quiet-3u3b (xn--quiet-3u3b)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--quiet-3u3b’
--2023-08-15 16:52:23--  http://xn--save-cookies-w19f/
Resolving xn--save-cookies-w19f (xn--save-cookies-w19f)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--save-cookies-w19f’
/root/cookies.txt: Scheme missing.
--2023-08-15 16:52:23--  http://xn--keep-session-cookies-ou2l/
Resolving xn--keep-session-cookies-ou2l (xn--keep-session-cookies-ou2l)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--keep-session-cookies-ou2l’
--2023-08-15 16:52:23--  http://xn--no-check-certificate-ou2l/
Resolving xn--no-check-certificate-ou2l (xn--no-check-certificate-ou2l)... failed: Name or service not known.
wget: unable to resolve host address ‘xn--no-check-certificate-ou2l’
--2023-08-15 16:52:23--  https://docs.google.com/uc?export=download&id=1a9XB3r83ZCFWLOHBp

In [ ]:
!unzip -qq open.zip

## Import

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00


In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16429 sha256=656644d205dc847dc2076fe385153d38b9b92b38d2cb9077993b8a735c911594
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
import os
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models # 이미지
from torchvision import transforms
from PIL import Image

from transformers import GPT2Tokenizer, GPT2Model # 텍스트

from tqdm.auto import tqdm
from efficientnet_pytorch import EfficientNet

## Dataset

In [ ]:
class VQADataset(Dataset):
    def __init__(self, df, tokenizer, transform, img_path, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.transform = transform
        self.img_path = img_path
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg') # 이미지
        image = Image.open(img_name).convert('RGB')
        image = self.transform(image)

        question = row['question'] # 질문
        question = self.tokenizer.encode_plus(
            question,
            truncation=True,
            add_special_tokens=True,
            max_length=32,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        if not self.is_test:
            answer = row['answer'] # 답변
            answer = self.tokenizer.encode_plus(
                answer,
                max_length=32,
                padding='max_length',
                truncation=True,
                return_tensors='pt')
            return {
                'image': image.squeeze(),
                'question': question['input_ids'].squeeze(),
                'answer': answer['input_ids'].squeeze()
            }
        else:
            return {
                'image': image,
                'question': question['input_ids'].squeeze(),
            }

## Model

In [ ]:
class VQAModel(nn.Module):
    def __init__(self, vocab_size):
        super(VQAModel, self).__init__()
        self.vocab_size = vocab_size
        self.efficient = EfficientNet.from_pretrained('efficientnet-b0')
        #self.resnet = models.resnet50(pretrained=True)
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.gpt2.resize_token_embeddings(vocab_size) # 추가한 [PAD] 토큰 반영

        combined_features_size = 1000 + self.gpt2.config.hidden_size # efficient 출력 차원 + gpt2 출력 차원
        self.classifier = nn.Linear(combined_features_size, vocab_size)

    def forward(self, images, question):
        image_features = self.efficient(images)
        image_features = image_features.view(image_features.size(0),-1)

        outputs = self.gpt2(question)
        output_features = outputs.last_hidden_state # [batch, sequence, hidden]

        image_features = image_features.unsqueeze(1).expand(-1, output_features.size(1),-1) # [batch, sequence, 1000]

        combined = torch.cat([image_features, output_features], dim=-1) # [batch, sequence, 1000+hidden]
        output = self.classifier(combined) # [batch, vocab_size]
        return output

## DataLoader

In [ ]:
# 데이터 불러오기
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')
train_img_path = '/content/image/train'
test_img_path = '/content/image/test'

# dataset & dataloader
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
vocab_size = len(tokenizer)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = VQADataset(train_df, tokenizer, transform, train_img_path, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Train & Inference

In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for data in tqdm(loader, total=len(loader)):
        images = data['image'].to(device)
        question = data['question'].to(device)
        answer = data['answer'].to(device)

        optimizer.zero_grad()

        outputs = model(images, question)

        # output: [batch, sequence, vocab], answer : [batch, sequence]
        loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(loader)
    return avg_loss

In [ ]:
def inference(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for data in tqdm(loader, total=len(loader)):
            images = data['image'].to(device)
            question = data['question'].to(device)

            outputs = model(images, question) # [batch, sequence, vocab]

            _, pred = torch.max(outputs, dim=2) # values, indices = _, pred
            preds.extend(pred.cpu().numpy())

    return preds

## Run

In [ ]:
# device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"current device is {device}")

# Model
model = VQAModel(vocab_size).to(device)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):
    avg_loss = train(model, train_loader, optimizer, criterion)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")
    torch.save({
      'model': model,
      'optimizer': optimizer,
      'epoch': epoch
      }, f'baseline_{epoch+1}.pth')

current device is cuda


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 26.6MB/s]


Loaded pretrained weights for efficientnet-b0


  0%|          | 0/5618 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.2370


  0%|          | 0/5618 [00:00<?, ?it/s]

Epoch: 2, Loss: 0.1694


  0%|          | 0/5618 [00:00<?, ?it/s]

Epoch: 3, Loss: 0.1573


## Post-Processing

In [ ]:
# Dataset & DataLoader
test_dataset = VQADataset(test_df, tokenizer, transform, test_img_path, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# inference
preds = inference(model, test_loader)

no_pad_output = []
for pred in preds:
    output = pred[pred != 50257] # [PAD] token 제외
    no_pad_output.append(tokenizer.decode(output).strip()) # 토큰 id -> 토큰

  0%|          | 0/633 [00:00<?, ?it/s]

## Submission

In [ ]:
sample_submission['answer'] = no_pad_output
sample_submission.to_csv('/content/submission1.csv', index=False)

In [ ]:
submission = pd.read_csv('/content/submission1.csv')

In [ ]:
submission

,ID,answer
0,TEST_00000,yes
1,TEST_00001,be
2,TEST_00002,s
3,TEST_00003,wood
4,TEST_00004,no
...,...,...
40474,TEST_40474,red
40475,TEST_40475,b
40476,TEST_40476,no
40477,TEST_40477,ele


In [ ]:
PATH = '/content/EfficientNetb0_VisualBERT.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
#from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!cp /content/EfficientNetb0_GPT2.pth /content/drive/MyDrive/

In [ ]:
#!cp /content/baseline_1.pth /content/drive/MyDrive/

In [ ]:
#!cp /content/baseline_2.pth /content/drive/MyDrive/

In [ ]:
#!cp /content/baseline_3.pth /content/drive/MyDrive/